In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('/mnt/c/Users/ouyangkang/Desktop/任务单W250211025测序数据-20250305.xlsx')

/home/ouyangkang/software/miniconda3/envs/sc/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
data.head()

,状态,加测样品编号,生产样品编号,样品名称,物种,数据文件名称,归属方向,任务单号,SN号,碱基质量体系,...,samNCountRead2,samNCountTotal,samGC,samQ10,samQ20,samQ30Read1,samQ30Read2,samQ30Total,Average sequencing depth (X),Coverage at least 20X (%)
0,已交付,NaN,Y00785N9-1,Y00785N9-1,时空组学-human,DP8480013523BR_L01_1~4,超级细胞研究所,W250211025,Y00785N9,Q4,...,0.01,0.01,49.20,99.99,97.27,89.62,91.46,90.96,NaN,NaN
1,已交付,NaN,Y00785N9-2,Y00785N9-2,时空组学-human,DP8480013522BR_L01_5~8,超级细胞研究所,W250211025,Y00785N9,Q4,...,0.00,0.00,51.24,100.00,98.03,92.18,92.92,92.72,NaN,NaN
2,已交付,NaN,Y00785NC-1,Y00785NC-1,时空组学-human,DP8480013502BR_L01_9~12,超级细胞研究所,W250211025,Y00785NC,Q4,...,0.01,0.01,49.93,99.99,96.83,87.57,90.80,89.92,NaN,NaN
3,已交付,NaN,Y00785NC-2,Y00785NC-2,时空组学-human,DP8480014360TR_L01_13~16,超级细胞研究所,W250211025,Y00785NC,Q4,...,0.00,0.00,47.26,100.00,96.77,87.25,90.90,89.91,NaN,NaN
4,已交付,NaN,Y00785ND-1,Y00785ND-1,时空组学-human,DP8480013502BR_L01_17&19~21&23~24&33,超级细胞研究所,W250211025,Y00785ND,Q4,...,0.01,0.01,49.58,99.99,96.35,86.68,89.13,88.47,NaN,NaN


In [68]:
output_df = pd.DataFrame(columns=['EntityID', 'SN', 'FASTQ1', 'FASTQ2', 'MASK', 'ImageTar', 'ImagePreDir', 'IPR', 'SplitNum', 'Tissue', 'CIDStart', 'CIDLength', 'MIDStart', 'MIDLength', 'rRNARemove', 'LargeChip', 'Reference',
                                  'CellSpecies', 'CellSample'])

In [65]:
data = data[(data['状态'] != '终止') & (data['状态'].notna())]

In [67]:
data.shape

(13, 65)

In [70]:
from pathlib import Path

index_num = 0

for index, row in data.iterrows():
    dir_path = Path(row['二级路径'])
    barcode_list = row['Barcode号'].split('&')
    lane = row['Lane']
    seq_chip = row['芯片号']
    sample_id = row['样品名称']
    SN = row['SN号']

    if len(barcode_list) == 1:
        min_num = int(barcode_list[0].split('~')[0])
        max_num = int(barcode_list[0].split('~')[-1])
        for num in range(min_num, max_num+1):
            fq1_path = dir_path / f'{seq_chip}_{lane}_{num}_1.fq.gz'
            fq2_path = dir_path / f'{seq_chip}_{lane}_{num}_2.fq.gz'

            result_list = [sample_id, SN, fq1_path, fq2_path, f'/Files/RawData/{SN}.barcodeToPos.h5', None, None, None, 16, 'cell', 0, 25, 25, 10, 'FALSE', 'FALSE', 'human', 'human', 'pbmc']
            output_df.loc[len(output_df)] = result_list
    else:
        for num_list in barcode_list:
            min_num = int(num_list.split('~')[0])
            max_num = int(num_list.split('~')[-1])
            for num in range(min_num, max_num+1):
                fq1_path = dir_path / f'{seq_chip}_{lane}_{num}_1.fq.gz'
                fq2_path = dir_path / f'{seq_chip}_{lane}_{num}_2.fq.gz'
                result_list = [sample_id, SN, fq1_path, fq2_path, f'/Files/RawData/{SN}.barcodeToPos.h5', None, None, None, 16, 'cell', 0, 25, 25, 10, 'FALSE', 'FALSE', 'human', 'human', 'pbmc']
                output_df.loc[len(output_df)] = result_list

In [72]:
output_df.to_csv('0305_ascites_UMI.csv', index=None)

In [42]:
output_df.columns

Index(['SN', 'Sample_name', 'FQ1', 'FQ2'], dtype='object')